In [1]:
import pickle
import sqlite3
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

model = model.cuda()

In [5]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../data/BCU_database/bcu_caption.db")
Photo = pd.read_sql_query("SELECT * from Photo", con)

# Verify that result of SQL query is stored in the dataframe
Photo.head()

,id,bcu_id,title,title_en,caption,year_from,year_to,photographer_name,fund_name,locality,theme,filename,title_en_preprocessed,caption_preprocessed
0,1,ALBL00222,"Singine, Sense, Düdingen, moulin","Singine, Sense, Düdingen, mill",aerial view of the town.,1952,1952,None,Aéroport Lausanne Blécherette,Düdingen,Question de la modernité - urbanisme tardif,ALBL00222_2k_324w.jpg,"singine, sense, düdingen, mill",None
1,2,ALCU_00005,Inauguration de la plaque commémorative au mur...,Inauguration of the commemorative plaque on th...,crowds gather to watch the parade.,1900,1900,"Cuony, Xavier","Cuony, Xavier et Albert",Fribourg,Question religieuse - formation,ALCU_00005_2k_324w.jpg,inauguration of the commemorative plaque on th...,crowds gather to watch the parade.
2,3,ALCU_00009,"Marché à la Place Notre-Dame, autour de la fon...","Marché à la Place Notre-Dame, autour de la fou...","a city in the past, history.",1900,1900,"Cuony, Xavier","Cuony, Xavier et Albert",Fribourg,Question de la modernité - alimentation,ALCU_00009_2k_324w.jpg,"marché à la place notre-dame, autour de la fou...",None
3,4,ALCU_00033,"Patient du Dr Xavier Cuony, physicien de ville...","Patient of Dr Xavier Cuony, city physicist, Fr...",painting artist in his studio.,1895,1910,"Cuony, Xavier","Cuony, Xavier et Albert",Fribourg,Question de la modernité - santé,ALCU_00033_2k_324w.jpg,"patient of dr xavier cuony, city physicist,",painting artist in his studio.
4,5,ALNU_00015,"Homme âgé (Aloïs Catillaz), Felsenegg, Wünnewil","Elderly (Aloïs Catillaz), Felsenegg, Wünnewil",portrait of a man sitting on a bench.,1885,1900,"Nussbaumer, Aloïs","Nussbaumer, Aloïs",Wünnewil,Question sociale,ALNU_00015_2k_324w.jpg,"elderly (aloïs catillaz), felsenegg, wünnewil",a man sitting on a bench.


In [ ]:
def translate(sentence):
    input_ids = tokenizer(sentence, return_tensors="pt").input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, num_beams=5, num_return_sequences=1)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [ ]:
Photo['title_en'] = Photo['title'].apply(lambda x: translate(x) if pd.notna(x) else '')

In [ ]:
Photo.to_sql("Photo",con,if_exists='replace',index=False, index_label="id")